In [3]:
import torch
import torchvision
from torch.utils.data import DataLoader, random_split, Subset, Dataset
from torchvision.transforms import ToTensor
import torch.nn as nn

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd "drive/MyDrive/Enappsys Data"

/content/drive/MyDrive/Enappsys Data


In [10]:
from sklearn.preprocessing import StandardScaler
import time

In [11]:
from sklearn.metrics import mean_squared_error

### Creating an abstract class of the Dataset class in torch module. Modifying it according to the data we require.

In [12]:
class CustomDataset(Dataset):
    def __init__(self, filename):
        """
        This is extraction step of the data processing
        """
        file_out = pd.read_csv(filename)
        x = file_out.iloc[:, :-1].values
        y = file_out.iloc[:, -1].values

        sc = StandardScaler()
        x =  sc.fit_transform(x)

        """
        This is the transformation step of data
        """
        self.X = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]




In [13]:
dataset = CustomDataset('train.csv')

### Can't create the same for test dataset yet. because it doesnt have any **Y** variable

### Dataset is created.Now we will create the DataLoader to be fed in the network. This is the Loading step of the data. ETL is completed after this

In [14]:
# Define the train/val split
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])

In [15]:
len(train_set)

16430

In [16]:
x, y = next(iter(train_set))

In [17]:
x.shape

torch.Size([1387])

In [18]:
y

tensor(0.)

In [19]:
# Loading the dataset using data loader and making mini-batches
train_loader = DataLoader(
    dataset=train_set,
    batch_size=64,
    shuffle=True
)

val_loader = DataLoader(
    dataset=val_set,
    batch_size=64,
    shuffle=True
)

In [ ]:
x_batch, y_batch = next(iter(train_loader))

In [ ]:
x_batch.shape

torch.Size([64, 1387])

In [ ]:
y_batch.shape

torch.Size([64])

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Implementing Feed forward network

In [15]:
input_layer = 1387
hidden_layer=512
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adamax(mlp.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
# mlp.to(device)

In [ ]:
train_losses = []
train_accs=[]

epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # inputs.to(device)
      # targets.to(device)

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs, targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([30])) that is different to the input size (torch.Size([30, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


time taken for epoch 1 is 3.24
Starting epoch 2
time taken for epoch 2 is 3.23
Starting epoch 3
time taken for epoch 3 is 4.00
Starting epoch 4
time taken for epoch 4 is 3.02
Starting epoch 5
time taken for epoch 5 is 3.24
Starting epoch 6
time taken for epoch 6 is 3.63
Starting epoch 7
time taken for epoch 7 is 4.27
Starting epoch 8
time taken for epoch 8 is 3.33
Starting epoch 9
time taken for epoch 9 is 3.38
Starting epoch 10
time taken for epoch 10 is 4.11
Starting epoch 11
time taken for epoch 11 is 3.95
Starting epoch 12
time taken for epoch 12 is 3.41
Starting epoch 13
time taken for epoch 13 is 3.42
Starting epoch 14
time taken for epoch 14 is 4.44
Starting epoch 15
time taken for epoch 15 is 3.48
Starting epoch 16
time taken for epoch 16 is 3.49
Starting epoch 17
time taken for epoch 17 is 3.68
Starting epoch 18
time taken for epoch 18 is 4.20
Starting epoch 19
time taken for epoch 19 is 3.46
Starting epoch 20
time taken for epoch 20 is 3.54
Training process has finished. Time

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# sets the model in evaluation mode
mlp.eval()

train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 4234928.25745252
--------------------------------------------------
Validation:
Validation MSE 3997192.331822769


##Model 2

In [47]:
input_layer = 1387
hidden_layer=512
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_losses = []
train_accs=[]

epochs=50

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 4.65
Starting epoch 2
time taken for epoch 2 is 4.59
Starting epoch 3
time taken for epoch 3 is 4.06
Starting epoch 4
time taken for epoch 4 is 4.38
Starting epoch 5
train MSE after epoch 5 is 124730.64620978803
time taken for epoch 5 is 5.09
Starting epoch 6
time taken for epoch 6 is 4.77
Starting epoch 7
time taken for epoch 7 is 5.69
Starting epoch 8
time taken for epoch 8 is 4.67
Starting epoch 9
time taken for epoch 9 is 4.79
Starting epoch 10
train MSE after epoch 10 is 129043.04725480318
time taken for epoch 10 is 5.76
Starting epoch 11
time taken for epoch 11 is 4.71
Starting epoch 12
time taken for epoch 12 is 5.10
Starting epoch 13
time taken for epoch 13 is 5.28
Starting epoch 14
time taken for epoch 14 is 4.61
Starting epoch 15
train MSE after epoch 15 is 106464.56447823152
time taken for epoch 15 is 5.64
Starting epoch 16
time taken for epoch 16 is 4.85
Starting epoch 17
time taken for epoch 17 is 4.89
Starting epoch 18
time taken

In [ ]:
# sets the model in evaluation mode
mlp.eval()

train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 11635.30885145958
--------------------------------------------------
Validation:
Validation MSE 61558.17472441997


## Model 3

In [ ]:
input_layer = 1387
hidden_layer=512
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.002, weight_decay=0.002)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]

epochs=50

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 3.58
Starting epoch 2
time taken for epoch 2 is 3.06
Starting epoch 3
time taken for epoch 3 is 3.08
Starting epoch 4
time taken for epoch 4 is 3.30
Starting epoch 5
train MSE after epoch 5 is 115756.8885570178
time taken for epoch 5 is 3.50
Starting epoch 6
time taken for epoch 6 is 3.09
Starting epoch 7
time taken for epoch 7 is 3.09
Starting epoch 8
time taken for epoch 8 is 3.44
Starting epoch 9
time taken for epoch 9 is 3.40
Starting epoch 10
train MSE after epoch 10 is 93728.80211051911
time taken for epoch 10 is 3.08
Starting epoch 11
time taken for epoch 11 is 3.07
Starting epoch 12
time taken for epoch 12 is 3.40
Starting epoch 13
time taken for epoch 13 is 3.40
Starting epoch 14
time taken for epoch 14 is 3.10
Starting epoch 15
train MSE after epoch 15 is 81957.79833478681
time taken for epoch 15 is 3.09
Starting epoch 16
time taken for epoch 16 is 3.48
Starting epoch 17
time taken for epoch 17 is 3.24
Starting epoch 18
time taken fo

In [ ]:
# sets the model in evaluation mode
mlp.eval()

train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 13765.324907054272
--------------------------------------------------
Validation:
Validation MSE 56628.916922751094


##Model 4

In [ ]:
input_layer = 1387
hidden_layer=512
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.002, weight_decay=0.005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=80

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 9.62
Starting epoch 2
time taken for epoch 2 is 9.56
Starting epoch 3
time taken for epoch 3 is 9.69
Starting epoch 4
time taken for epoch 4 is 9.40
Starting epoch 5
train MSE after epoch 5 is 14819.719137506918
time taken for epoch 5 is 10.20
Starting epoch 6
time taken for epoch 6 is 10.12
Starting epoch 7
time taken for epoch 7 is 9.88
Starting epoch 8
time taken for epoch 8 is 10.02
Starting epoch 9
time taken for epoch 9 is 10.75
Starting epoch 10
train MSE after epoch 10 is 15348.661362072591
time taken for epoch 10 is 10.78
Starting epoch 11
time taken for epoch 11 is 10.76
Starting epoch 12
time taken for epoch 12 is 10.08
Starting epoch 13
time taken for epoch 13 is 10.42
Starting epoch 14
time taken for epoch 14 is 10.55
Starting epoch 15
train MSE after epoch 15 is 14955.012152457894
time taken for epoch 15 is 10.21
Starting epoch 16
time taken for epoch 16 is 9.63
Starting epoch 17
time taken for epoch 17 is 10.17
Starting epoch 18

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 2605.475234472611
--------------------------------------------------
Validation:
Validation MSE 47154.73558715159


##Model 5

In [ ]:
input_layer = 1387
hidden_layer=512
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.RMSprop(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.001,
#                      step_size_up=2000, mode='triangular2')
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 3.35
Starting epoch 2
time taken for epoch 2 is 3.99
Starting epoch 3
time taken for epoch 3 is 3.80
Starting epoch 4
time taken for epoch 4 is 3.22
Starting epoch 5
time taken for epoch 5 is 3.18
Starting epoch 6
time taken for epoch 6 is 4.01
Starting epoch 7
time taken for epoch 7 is 3.70
Starting epoch 8
time taken for epoch 8 is 3.20
Starting epoch 9
time taken for epoch 9 is 3.23
Starting epoch 10
time taken for epoch 10 is 4.07
Starting epoch 11
time taken for epoch 11 is 3.65
Starting epoch 12
time taken for epoch 12 is 3.21
Starting epoch 13
time taken for epoch 13 is 3.16
Starting epoch 14
time taken for epoch 14 is 4.14
Starting epoch 15
time taken for epoch 15 is 3.52
Starting epoch 16
time taken for epoch 16 is 3.24
Starting epoch 17
time taken for epoch 17 is 3.23
Starting epoch 18
time taken for epoch 18 is 4.27
Starting epoch 19
time taken for epoch 19 is 3.48
Starting epoch 20
time taken for epoch 20 is 3.31
Training process h

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 50450.149413284016
--------------------------------------------------
Validation:
Validation MSE 103541.2527976258


In [ ]:
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  # mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  # train_mse.append(mse)

In [ ]:
predict_batch

tensor([[ 4.1676e+03],
        [ 2.0682e+01],
        [ 2.8995e+01],
        [ 3.1990e+00],
        [ 9.0011e+01],
        [ 1.3886e+00],
        [ 3.3306e+03],
        [ 1.0512e+01],
        [-4.3013e+00],
        [ 6.7146e+03],
        [-1.1334e+01],
        [-5.8078e+00],
        [ 1.2085e+01],
        [ 7.3597e+03],
        [-5.5015e+00],
        [ 4.1414e+00],
        [ 4.4336e+02],
        [ 4.8140e+02],
        [ 8.3415e+02],
        [ 5.0055e+00],
        [ 2.6227e+01],
        [ 1.6758e+03],
        [-7.4917e+00],
        [ 6.1105e+00],
        [ 6.7051e+02],
        [ 5.2950e+03],
        [ 4.5567e+01],
        [ 8.8066e+02],
        [ 5.7776e+02],
        [-4.1270e+00],
        [ 2.0154e+02],
        [ 5.7269e+03],
        [ 2.2047e+03],
        [ 8.1648e+01],
        [ 4.2944e+00],
        [-6.5415e+00],
        [ 5.1655e-01],
        [-5.7766e+00],
        [ 5.7695e+03],
        [ 3.8361e+02],
        [ 6.8106e-01],
        [ 5.4025e+00],
        [ 1.9399e+01],
        [ 1

In [ ]:
input_batch, target_batch = next(iter(train_loader))

In [ ]:
input_batch.flatten(1,-1).shape

torch.Size([64, 1387])

In [ ]:
target_batch.shape

torch.Size([64])

In [ ]:
mlp

MLP(
  (layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1387, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): ReLU()
    (7): Linear(in_features=512, out_features=1, bias=True)
  )
)

In [ ]:
predict_batch

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)

In [ ]:
!pip install skorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 4.2 MB/s eta 0:00:00


In [ ]:
 import skorch

In [ ]:
from sklearn.model_selection import GridSearchCV

## Tuning batch sie, epochs, optimizer, lr

In [ ]:
model = skorch.NeuralNetRegressor(
    module=mlp
    # max_epochs=150,
    # batch_size=10
)

In [ ]:
param_grid = {
    'optimizer': [optim.RMSprop, optim.Adagrad, optim.Adadelta,
                  optim.Adam, optim.Adamax, optim.NAdam],
     'batch_size': [32, 64, 128],
    'optimizer__lr': [0.0001, 0.0005, 0.001, 0.002, 0.003],
    'optimizer__momentum': [0.0, 0.2, 0.4, 0.6, 0.8, 0.9],
    'max_epochs': [50,80,100]
}


In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

## Making X and y labels

In [ ]:
# Initialize an empty list to store the individual tensors
record_tensors_ip = []
record_tensors_op=[]

# Iterate over the data loader and collect the tensors
for ip, op in train_set:
    record_tensors_ip.append(ip)
    record_tensors_op.append(op)

In [ ]:
X = torch.stack(record_tensors_ip)
y= torch.stack(record_tensors_op)

In [ ]:
X.shape

torch.Size([16430, 1387])

In [ ]:
y=y.reshape(-1,1)

In [ ]:
y.shape

torch.Size([16430, 1])

## Fitting the model

In [ ]:
model = skorch.NeuralNetRegressor(
    module=mlp,
    optimizer= optim.Adam,
    criterion=nn.MSELoss(),
    # max_epochs=50,
    batch_size=10,
    verbose=False
)

# model = NeuralNetClassifier(
#     PimaClassifier,
#     criterion=nn.BCELoss,
#     optimizer=optim.Adamax,
#     max_epochs=100,
#     batch_size=10,
#     verbose=False
# )

In [ ]:
param_grid = {
    # 'optimizer': [optim.RMSprop, optim.Adagrad, optim.Adadelta,
    #               optim.Adam, optim.Adamax, optim.NAdam],
    # 'batch_size': [32, 64, 128],
    # 'optimizer__lr': [0.0001, 0.0005, 0.001, 0.002, 0.003],
    # 'optimizer__momentum': [0.0, 0.2, 0.4, 0.6, 0.8, 0.9],
    'max_epochs': [50,80,100]
}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=2)

In [ ]:
grid_result = grid.fit(X, y)

KeyboardInterrupt: ignored

###Hyperparameter Tuning

###Initializing the number of layers

##layers=3

In [ ]:
input_layer = 1387
hidden_layer=512
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 13.37
Starting epoch 2
time taken for epoch 2 is 5.06
Starting epoch 3
time taken for epoch 3 is 5.22
Starting epoch 4
time taken for epoch 4 is 5.41
Starting epoch 5
time taken for epoch 5 is 4.87
Starting epoch 6
time taken for epoch 6 is 5.99
Starting epoch 7
time taken for epoch 7 is 4.77
Starting epoch 8
time taken for epoch 8 is 4.77
Starting epoch 9
time taken for epoch 9 is 5.97
Starting epoch 10
time taken for epoch 10 is 4.78
Starting epoch 11
time taken for epoch 11 is 5.04
Starting epoch 12
time taken for epoch 12 is 5.69
Starting epoch 13
time taken for epoch 13 is 4.77
Starting epoch 14
time taken for epoch 14 is 5.30
Starting epoch 15
time taken for epoch 15 is 5.32
Starting epoch 16
time taken for epoch 16 is 4.75
Starting epoch 17
time taken for epoch 17 is 5.74
Starting epoch 18
time taken for epoch 18 is 5.32
Starting epoch 19
time taken for epoch 19 is 4.77
Starting epoch 20
time taken for epoch 20 is 6.37
Training process 

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 46039.599349818585
--------------------------------------------------
Validation:
Validation MSE 82865.26028978653


##layers=4

In [ ]:
input_layer = 1387
hidden_layer=512
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 5.57
Starting epoch 2
time taken for epoch 2 is 4.70
Starting epoch 3
time taken for epoch 3 is 4.52
Starting epoch 4
time taken for epoch 4 is 6.16
Starting epoch 5
time taken for epoch 5 is 4.91
Starting epoch 6
time taken for epoch 6 is 4.86
Starting epoch 7
time taken for epoch 7 is 6.05
Starting epoch 8
time taken for epoch 8 is 4.81
Starting epoch 9
time taken for epoch 9 is 5.21
Starting epoch 10
time taken for epoch 10 is 5.67
Starting epoch 11
time taken for epoch 11 is 4.83
Starting epoch 12
time taken for epoch 12 is 5.80
Starting epoch 13
time taken for epoch 13 is 6.26
Starting epoch 14
time taken for epoch 14 is 5.04
Starting epoch 15
time taken for epoch 15 is 6.02
Starting epoch 16
time taken for epoch 16 is 5.24
Starting epoch 17
time taken for epoch 17 is 5.48
Starting epoch 18
time taken for epoch 18 is 5.42
Starting epoch 19
time taken for epoch 19 is 4.85
Starting epoch 20
time taken for epoch 20 is 5.95
Training process h

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 32693.866327709165
--------------------------------------------------
Validation:
Validation MSE 72993.5774151891


##layers=5

In [ ]:
input_layer = 1387
hidden_layer=512
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 6.69
Starting epoch 2
time taken for epoch 2 is 7.45
Starting epoch 3
time taken for epoch 3 is 6.28
Starting epoch 4
time taken for epoch 4 is 7.24
Starting epoch 5
time taken for epoch 5 is 6.50
Starting epoch 6
time taken for epoch 6 is 7.05
Starting epoch 7
time taken for epoch 7 is 6.80
Starting epoch 8
time taken for epoch 8 is 6.79
Starting epoch 9
time taken for epoch 9 is 7.00
Starting epoch 10
time taken for epoch 10 is 6.47
Starting epoch 11
time taken for epoch 11 is 7.25
Starting epoch 12
time taken for epoch 12 is 6.29
Starting epoch 13
time taken for epoch 13 is 7.51
Starting epoch 14
time taken for epoch 14 is 6.29
Starting epoch 15
time taken for epoch 15 is 7.48
Starting epoch 16
time taken for epoch 16 is 6.32
Starting epoch 17
time taken for epoch 17 is 7.49
Starting epoch 18
time taken for epoch 18 is 6.32
Starting epoch 19
time taken for epoch 19 is 7.48
Starting epoch 20
time taken for epoch 20 is 6.29
Training process h

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 8933.646734787882
--------------------------------------------------
Validation:
Validation MSE 54878.90346602896


##layers=6

In [ ]:
input_layer = 1387
hidden_layer=512
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 11.94
Starting epoch 2
time taken for epoch 2 is 6.43
Starting epoch 3
time taken for epoch 3 is 7.94
Starting epoch 4
time taken for epoch 4 is 7.87
Starting epoch 5
time taken for epoch 5 is 8.82
Starting epoch 6
time taken for epoch 6 is 9.00
Starting epoch 7
time taken for epoch 7 is 7.81
Starting epoch 8
time taken for epoch 8 is 8.94
Starting epoch 9
time taken for epoch 9 is 7.92
Starting epoch 10
time taken for epoch 10 is 9.05
Starting epoch 11
time taken for epoch 11 is 8.67
Starting epoch 12
time taken for epoch 12 is 8.28
Starting epoch 13
time taken for epoch 13 is 9.12
Starting epoch 14
time taken for epoch 14 is 7.89
Starting epoch 15
time taken for epoch 15 is 9.10
Starting epoch 16
time taken for epoch 16 is 8.60
Starting epoch 17
time taken for epoch 17 is 9.57
Starting epoch 18
time taken for epoch 18 is 9.16
Starting epoch 19
time taken for epoch 19 is 8.03
Starting epoch 20
time taken for epoch 20 is 9.07
Training process 

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 35689.51569290848
--------------------------------------------------
Validation:
Validation MSE 77122.47669966245


## Proceeding with 5 layers. tuning the neurons

##neurons=1024

In [ ]:
input_layer = 1387
hidden_layer=1024
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 18.51
Starting epoch 2
time taken for epoch 2 is 16.71
Starting epoch 3
time taken for epoch 3 is 17.87
Starting epoch 4
time taken for epoch 4 is 22.06
Starting epoch 5
time taken for epoch 5 is 28.88
Starting epoch 6
time taken for epoch 6 is 22.17
Starting epoch 7
time taken for epoch 7 is 32.36
Starting epoch 8
time taken for epoch 8 is 23.20
Starting epoch 9
time taken for epoch 9 is 20.93
Starting epoch 10
time taken for epoch 10 is 21.71
Starting epoch 11
time taken for epoch 11 is 21.26
Starting epoch 12
time taken for epoch 12 is 21.63
Starting epoch 13
time taken for epoch 13 is 21.35
Starting epoch 14
time taken for epoch 14 is 20.99
Starting epoch 15
time taken for epoch 15 is 21.96
Starting epoch 16
time taken for epoch 16 is 20.76
Starting epoch 17
time taken for epoch 17 is 22.39
Starting epoch 18
time taken for epoch 18 is 22.43
Starting epoch 19
time taken for epoch 19 is 22.01
Starting epoch 20
time taken for epoch 20 is 21.0

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 46151.40270158622
--------------------------------------------------
Validation:
Validation MSE 85679.74331944234


##neurons=768

In [ ]:
input_layer = 1387
hidden_layer=768
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 9.33
Starting epoch 2
time taken for epoch 2 is 10.49
Starting epoch 3
time taken for epoch 3 is 11.08
Starting epoch 4
time taken for epoch 4 is 12.89
Starting epoch 5
time taken for epoch 5 is 11.78
Starting epoch 6
time taken for epoch 6 is 12.29
Starting epoch 7
time taken for epoch 7 is 12.54
Starting epoch 8
time taken for epoch 8 is 12.51
Starting epoch 9
time taken for epoch 9 is 12.67
Starting epoch 10
time taken for epoch 10 is 13.04
Starting epoch 11
time taken for epoch 11 is 13.43
Starting epoch 12
time taken for epoch 12 is 12.26
Starting epoch 13
time taken for epoch 13 is 12.02
Starting epoch 14
time taken for epoch 14 is 12.62
Starting epoch 15
time taken for epoch 15 is 12.63
Starting epoch 16
time taken for epoch 16 is 12.62
Starting epoch 17
time taken for epoch 17 is 12.64
Starting epoch 18
time taken for epoch 18 is 12.64
Starting epoch 19
time taken for epoch 19 is 12.46
Starting epoch 20
time taken for epoch 20 is 12.00

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 42457.44449924301
--------------------------------------------------
Validation:
Validation MSE 78065.58092123545


##neurons=512

In [ ]:
input_layer = 1387
hidden_layer=512
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 8.19
Starting epoch 2
time taken for epoch 2 is 6.34
Starting epoch 3
time taken for epoch 3 is 8.48
Starting epoch 4
time taken for epoch 4 is 11.43
Starting epoch 5
time taken for epoch 5 is 8.22
Starting epoch 6
time taken for epoch 6 is 8.62
Starting epoch 7
time taken for epoch 7 is 7.24
Starting epoch 8
time taken for epoch 8 is 9.75
Starting epoch 9
time taken for epoch 9 is 9.11
Starting epoch 10
time taken for epoch 10 is 9.50
Starting epoch 11
time taken for epoch 11 is 7.49
Starting epoch 12
time taken for epoch 12 is 6.14
Starting epoch 13
time taken for epoch 13 is 7.23
Starting epoch 14
time taken for epoch 14 is 6.27
Starting epoch 15
time taken for epoch 15 is 6.90
Starting epoch 16
time taken for epoch 16 is 6.64
Starting epoch 17
time taken for epoch 17 is 6.59
Starting epoch 18
time taken for epoch 18 is 7.62
Starting epoch 19
time taken for epoch 19 is 6.50
Starting epoch 20
time taken for epoch 20 is 7.13
Training process 

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 34139.81565825409
--------------------------------------------------
Validation:
Validation MSE 72279.50304992679


##neurons=256

In [ ]:
input_layer = 1387
hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(hidden_layer, hidden_layer, bias=True),
        nn.ReLU(),
        nn.Linear(hidden_layer, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 2.65
Starting epoch 2
time taken for epoch 2 is 3.01
Starting epoch 3
time taken for epoch 3 is 3.37
Starting epoch 4
time taken for epoch 4 is 2.76
Starting epoch 5
time taken for epoch 5 is 2.75
Starting epoch 6
time taken for epoch 6 is 2.72
Starting epoch 7
time taken for epoch 7 is 3.26
Starting epoch 8
time taken for epoch 8 is 3.31
Starting epoch 9
time taken for epoch 9 is 2.64
Starting epoch 10
time taken for epoch 10 is 2.65
Starting epoch 11
time taken for epoch 11 is 2.67
Starting epoch 12
time taken for epoch 12 is 3.39
Starting epoch 13
time taken for epoch 13 is 3.26
Starting epoch 14
time taken for epoch 14 is 2.69
Starting epoch 15
time taken for epoch 15 is 2.69
Starting epoch 16
time taken for epoch 16 is 2.70
Starting epoch 17
time taken for epoch 17 is 3.46
Starting epoch 18
time taken for epoch 18 is 3.12
Starting epoch 19
time taken for epoch 19 is 2.71
Starting epoch 20
time taken for epoch 20 is 2.74
Training process h

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 35062.252907045535
--------------------------------------------------
Validation:
Validation MSE 74871.90164827972


## Dynamic

In [ ]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, 786, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(786, 512, bias=True),
        nn.ReLU(),
        nn.Linear(512, 384, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(384, 256, bias=True),
        nn.ReLU(),
        nn.Linear(256, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 5.71
Starting epoch 2
time taken for epoch 2 is 5.47
Starting epoch 3
time taken for epoch 3 is 5.19
Starting epoch 4
time taken for epoch 4 is 6.57
Starting epoch 5
time taken for epoch 5 is 5.54
Starting epoch 6
time taken for epoch 6 is 6.43
Starting epoch 7
time taken for epoch 7 is 5.61
Starting epoch 8
time taken for epoch 8 is 6.67
Starting epoch 9
time taken for epoch 9 is 6.06
Starting epoch 10
time taken for epoch 10 is 5.55
Starting epoch 11
time taken for epoch 11 is 6.36
Starting epoch 12
time taken for epoch 12 is 5.46
Starting epoch 13
time taken for epoch 13 is 6.53
Starting epoch 14
time taken for epoch 14 is 5.44
Starting epoch 15
time taken for epoch 15 is 6.30
Starting epoch 16
time taken for epoch 16 is 5.67
Starting epoch 17
time taken for epoch 17 is 5.65
Starting epoch 18
time taken for epoch 18 is 6.78
Starting epoch 19
time taken for epoch 19 is 5.55
Starting epoch 20
time taken for epoch 20 is 6.63
Training process h

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 26870.874534507133
--------------------------------------------------
Validation:
Validation MSE 65020.630912577784


##Dyanmic 2

In [ ]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, 512, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(512, 384, bias=True),
        nn.ReLU(),
        nn.Linear(384, 256, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(256, 128, bias=True),
        nn.ReLU(),
        nn.Linear(128, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 5.42
Starting epoch 2
time taken for epoch 2 is 3.54
Starting epoch 3
time taken for epoch 3 is 3.09
Starting epoch 4
time taken for epoch 4 is 4.23
Starting epoch 5
time taken for epoch 5 is 3.56
Starting epoch 6
time taken for epoch 6 is 3.63
Starting epoch 7
time taken for epoch 7 is 3.69
Starting epoch 8
time taken for epoch 8 is 3.85
Starting epoch 9
time taken for epoch 9 is 3.15
Starting epoch 10
time taken for epoch 10 is 3.13
Starting epoch 11
time taken for epoch 11 is 3.57
Starting epoch 12
time taken for epoch 12 is 3.89
Starting epoch 13
time taken for epoch 13 is 3.09
Starting epoch 14
time taken for epoch 14 is 3.14
Starting epoch 15
time taken for epoch 15 is 3.48
Starting epoch 16
time taken for epoch 16 is 3.92
Starting epoch 17
time taken for epoch 17 is 3.11
Starting epoch 18
time taken for epoch 18 is 3.09
Starting epoch 19
time taken for epoch 19 is 3.38
Starting epoch 20
time taken for epoch 20 is 3.98
Training process h

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 33544.97108113878
--------------------------------------------------
Validation:
Validation MSE 70359.13318612229


## Optimizer tuning

## Adam

In [ ]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, 786, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(786, 512, bias=True),
        nn.ReLU(),
        nn.Linear(512, 384, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(384, 256, bias=True),
        nn.ReLU(),
        nn.Linear(256, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 8.60
Starting epoch 2
time taken for epoch 2 is 7.77
Starting epoch 3
time taken for epoch 3 is 9.97
Starting epoch 4
time taken for epoch 4 is 10.25
Starting epoch 5
time taken for epoch 5 is 10.08
Starting epoch 6
time taken for epoch 6 is 7.89
Starting epoch 7
time taken for epoch 7 is 6.56
Starting epoch 8
time taken for epoch 8 is 5.73
Starting epoch 9
time taken for epoch 9 is 6.07
Starting epoch 10
time taken for epoch 10 is 6.31
Starting epoch 11
time taken for epoch 11 is 5.82
Starting epoch 12
time taken for epoch 12 is 6.85
Starting epoch 13
time taken for epoch 13 is 5.71
Starting epoch 14
time taken for epoch 14 is 6.75
Starting epoch 15
time taken for epoch 15 is 6.07
Starting epoch 16
time taken for epoch 16 is 6.93
Starting epoch 17
time taken for epoch 17 is 5.71
Starting epoch 18
time taken for epoch 18 is 7.16
Starting epoch 19
time taken for epoch 19 is 5.67
Starting epoch 20
time taken for epoch 20 is 6.89
Training process

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 24739.305803708332
--------------------------------------------------
Validation:
Validation MSE 63581.41863553035


## Adamax

In [ ]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, 786, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(786, 512, bias=True),
        nn.ReLU(),
        nn.Linear(512, 384, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(384, 256, bias=True),
        nn.ReLU(),
        nn.Linear(256, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adamax(mlp.parameters(), lr=0.0005, betas=(0.9,0.95))
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 9.50
Starting epoch 2
time taken for epoch 2 is 6.54
Starting epoch 3
time taken for epoch 3 is 6.35
Starting epoch 4
time taken for epoch 4 is 7.27
Starting epoch 5
time taken for epoch 5 is 6.43
Starting epoch 6
time taken for epoch 6 is 6.74
Starting epoch 7
time taken for epoch 7 is 6.56
Starting epoch 8
time taken for epoch 8 is 6.45
Starting epoch 9
time taken for epoch 9 is 6.70
Starting epoch 10
time taken for epoch 10 is 6.36
Starting epoch 11
time taken for epoch 11 is 6.86
Starting epoch 12
time taken for epoch 12 is 6.42
Starting epoch 13
time taken for epoch 13 is 7.23
Starting epoch 14
time taken for epoch 14 is 6.63
Starting epoch 15
time taken for epoch 15 is 7.79
Starting epoch 16
time taken for epoch 16 is 6.45
Starting epoch 17
time taken for epoch 17 is 6.86
Starting epoch 18
time taken for epoch 18 is 6.31
Starting epoch 19
time taken for epoch 19 is 7.02
Starting epoch 20
time taken for epoch 20 is 6.14
Training process h

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 31584.791863592964
--------------------------------------------------
Validation:
Validation MSE 71754.17680321276


## RMSProp

In [ ]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, 786, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(786, 512, bias=True),
        nn.ReLU(),
        nn.Linear(512, 384, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(384, 256, bias=True),
        nn.ReLU(),
        nn.Linear(256, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.RMSprop(mlp.parameters(), lr=0.0005, momentum=0.8)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 6.95
Starting epoch 2
time taken for epoch 2 is 6.87
Starting epoch 3
time taken for epoch 3 is 6.79
Starting epoch 4
time taken for epoch 4 is 7.44
Starting epoch 5
time taken for epoch 5 is 6.40
Starting epoch 6
time taken for epoch 6 is 7.43
Starting epoch 7
time taken for epoch 7 is 6.62
Starting epoch 8
time taken for epoch 8 is 7.66
Starting epoch 9
time taken for epoch 9 is 7.07
Starting epoch 10
time taken for epoch 10 is 7.39
Starting epoch 11
time taken for epoch 11 is 8.04
Starting epoch 12
time taken for epoch 12 is 7.11
Starting epoch 13
time taken for epoch 13 is 7.79
Starting epoch 14
time taken for epoch 14 is 6.93
Starting epoch 15
time taken for epoch 15 is 8.44
Starting epoch 16
time taken for epoch 16 is 7.06
Starting epoch 17
time taken for epoch 17 is 7.98
Starting epoch 18
time taken for epoch 18 is 7.47
Starting epoch 19
time taken for epoch 19 is 8.18
Starting epoch 20
time taken for epoch 20 is 8.05
Training process h

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 27448.095478300955
--------------------------------------------------
Validation:
Validation MSE 74746.31529967973


## Adagrad

In [ ]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, 786, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(786, 512, bias=True),
        nn.ReLU(),
        nn.Linear(512, 384, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(384, 256, bias=True),
        nn.ReLU(),
        nn.Linear(256, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adagrad(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 6.53
Starting epoch 2
time taken for epoch 2 is 7.09
Starting epoch 3
time taken for epoch 3 is 4.41
Starting epoch 4
time taken for epoch 4 is 5.02
Starting epoch 5
time taken for epoch 5 is 4.19
Starting epoch 6
time taken for epoch 6 is 3.92
Starting epoch 7
time taken for epoch 7 is 4.78
Starting epoch 8
time taken for epoch 8 is 4.45
Starting epoch 9
time taken for epoch 9 is 3.84
Starting epoch 10
time taken for epoch 10 is 4.46
Starting epoch 11
time taken for epoch 11 is 4.67
Starting epoch 12
time taken for epoch 12 is 3.97
Starting epoch 13
time taken for epoch 13 is 4.27
Starting epoch 14
time taken for epoch 14 is 4.94
Starting epoch 15
time taken for epoch 15 is 4.12
Starting epoch 16
time taken for epoch 16 is 4.58
Starting epoch 17
time taken for epoch 17 is 5.64
Starting epoch 18
time taken for epoch 18 is 4.05
Starting epoch 19
time taken for epoch 19 is 4.16
Starting epoch 20
time taken for epoch 20 is 5.00
Training process h

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 338017.01225521305
--------------------------------------------------
Validation:
Validation MSE 349593.2104329467


## NAdam

In [ ]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, 786, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(786, 512, bias=True),
        nn.ReLU(),
        nn.Linear(512, 384, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(384, 256, bias=True),
        nn.ReLU(),
        nn.Linear(256, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.NAdam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 9.91
Starting epoch 2
time taken for epoch 2 is 7.27
Starting epoch 3
time taken for epoch 3 is 6.83
Starting epoch 4
time taken for epoch 4 is 5.92
Starting epoch 5
time taken for epoch 5 is 6.82
Starting epoch 6
time taken for epoch 6 is 5.90
Starting epoch 7
time taken for epoch 7 is 6.40
Starting epoch 8
time taken for epoch 8 is 6.16
Starting epoch 9
time taken for epoch 9 is 6.05
Starting epoch 10
time taken for epoch 10 is 6.64
Starting epoch 11
time taken for epoch 11 is 5.94
Starting epoch 12
time taken for epoch 12 is 7.22
Starting epoch 13
time taken for epoch 13 is 5.95
Starting epoch 14
time taken for epoch 14 is 6.82
Starting epoch 15
time taken for epoch 15 is 5.86
Starting epoch 16
time taken for epoch 16 is 6.84
Starting epoch 17
time taken for epoch 17 is 5.86
Starting epoch 18
time taken for epoch 18 is 6.80
Starting epoch 19
time taken for epoch 19 is 6.23
Starting epoch 20
time taken for epoch 20 is 6.68
Training process h

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 38652.47938300552
--------------------------------------------------
Validation:
Validation MSE 74078.1636606325


## Learning Rate tuning

## LR=0.0001

In [ ]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, 786, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(786, 512, bias=True),
        nn.ReLU(),
        nn.Linear(512, 384, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(384, 256, bias=True),
        nn.ReLU(),
        nn.Linear(256, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 8.49
Starting epoch 2
time taken for epoch 2 is 7.46
Starting epoch 3
time taken for epoch 3 is 12.22
Starting epoch 4
time taken for epoch 4 is 12.61
Starting epoch 5
time taken for epoch 5 is 11.70
Starting epoch 6
time taken for epoch 6 is 9.45
Starting epoch 7
time taken for epoch 7 is 8.63
Starting epoch 8
time taken for epoch 8 is 9.56
Starting epoch 9
time taken for epoch 9 is 7.33
Starting epoch 10
time taken for epoch 10 is 9.54
Starting epoch 11
time taken for epoch 11 is 8.00
Starting epoch 12
time taken for epoch 12 is 6.14
Starting epoch 13
time taken for epoch 13 is 5.52
Starting epoch 14
time taken for epoch 14 is 6.42
Starting epoch 15
time taken for epoch 15 is 5.60
Starting epoch 16
time taken for epoch 16 is 7.59
Starting epoch 17
time taken for epoch 17 is 9.92
Starting epoch 18
time taken for epoch 18 is 8.79
Starting epoch 19
time taken for epoch 19 is 9.32
Starting epoch 20
time taken for epoch 20 is 8.80
Training proces

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 66768.10215552914
--------------------------------------------------
Validation:
Validation MSE 100484.27418224295


## LR = 0.0005

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    # for input_batch, target_batch in train_loader:
    #   predict_batch = mlp(input_batch)
    #   mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
    #   train_mse.append(mse)

    # if (epoch+1)%5==0:
    #     print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 6.08
Starting epoch 2
time taken for epoch 2 is 5.07
Starting epoch 3
time taken for epoch 3 is 5.51
Starting epoch 4
time taken for epoch 4 is 5.89
Starting epoch 5
time taken for epoch 5 is 5.35
Starting epoch 6
time taken for epoch 6 is 6.42
Starting epoch 7
time taken for epoch 7 is 5.75
Starting epoch 8
time taken for epoch 8 is 6.57
Starting epoch 9
time taken for epoch 9 is 5.36
Starting epoch 10
time taken for epoch 10 is 6.04
Starting epoch 11
time taken for epoch 11 is 5.78
Starting epoch 12
time taken for epoch 12 is 5.39
Starting epoch 13
time taken for epoch 13 is 6.39
Starting epoch 14
time taken for epoch 14 is 5.35
Starting epoch 15
time taken for epoch 15 is 6.39
Starting epoch 16
time taken for epoch 16 is 5.36
Starting epoch 17
time taken for epoch 17 is 6.22
Starting epoch 18
time taken for epoch 18 is 6.14
Starting epoch 19
time taken for epoch 19 is 5.35
Starting epoch 20
time taken for epoch 20 is 6.47
Training process h

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 25870.7025299591
--------------------------------------------------
Validation:
Validation MSE 65166.477607904024


## LR = 0.001

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 7.84
Starting epoch 2
time taken for epoch 2 is 6.57
Starting epoch 3
time taken for epoch 3 is 7.79
Starting epoch 4
time taken for epoch 4 is 7.50
Starting epoch 5
train MSE after epoch 5 is 191785.71222056026
time taken for epoch 5 is 8.32
Starting epoch 6
time taken for epoch 6 is 9.18
Starting epoch 7
time taken for epoch 7 is 7.52
Starting epoch 8
time taken for epoch 8 is 8.26
Starting epoch 9
time taken for epoch 9 is 7.30
Starting epoch 10
train MSE after epoch 10 is 163596.0986053243
time taken for epoch 10 is 8.77
Starting epoch 11
time taken for epoch 11 is 8.96
Starting epoch 12
time taken for epoch 12 is 7.39
Starting epoch 13
time taken for epoch 13 is 8.78
Starting epoch 14
time taken for epoch 14 is 7.55
Starting epoch 15
train MSE after epoch 15 is 125911.82953118725
time taken for epoch 15 is 8.33
Starting epoch 16
time taken for epoch 16 is 8.10
Starting epoch 17
time taken for epoch 17 is 7.61
Starting epoch 18
time taken 

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 28511.12896683742
--------------------------------------------------
Validation:
Validation MSE 68895.09800974824


## LR = 0.005

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 7.83
Starting epoch 2
time taken for epoch 2 is 6.64
Starting epoch 3
time taken for epoch 3 is 8.02
Starting epoch 4
time taken for epoch 4 is 8.50
Starting epoch 5
train MSE after epoch 5 is 136173.21998140207
time taken for epoch 5 is 8.81
Starting epoch 6
time taken for epoch 6 is 10.72
Starting epoch 7
time taken for epoch 7 is 8.34
Starting epoch 8
time taken for epoch 8 is 8.92
Starting epoch 9
time taken for epoch 9 is 9.41
Starting epoch 10
train MSE after epoch 10 is 109916.78320603375
time taken for epoch 10 is 8.19
Starting epoch 11
time taken for epoch 11 is 8.97
Starting epoch 12
time taken for epoch 12 is 9.39
Starting epoch 13
time taken for epoch 13 is 8.53
Starting epoch 14
time taken for epoch 14 is 9.27
Starting epoch 15
train MSE after epoch 15 is 94363.22653811541
time taken for epoch 15 is 9.35
Starting epoch 16
time taken for epoch 16 is 8.32
Starting epoch 17
time taken for epoch 17 is 9.18
Starting epoch 18
time taken

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 32829.117221064356
--------------------------------------------------
Validation:
Validation MSE 71275.00195161779


## LR = 0.01

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 8.21
Starting epoch 2
time taken for epoch 2 is 6.89
Starting epoch 3
time taken for epoch 3 is 8.16
Starting epoch 4
time taken for epoch 4 is 8.72
Starting epoch 5
train MSE after epoch 5 is 123082.86404113447
time taken for epoch 5 is 9.30
Starting epoch 6
time taken for epoch 6 is 9.21
Starting epoch 7
time taken for epoch 7 is 8.19
Starting epoch 8
time taken for epoch 8 is 9.53
Starting epoch 9
time taken for epoch 9 is 9.95
Starting epoch 10
train MSE after epoch 10 is 119040.74967316948
time taken for epoch 10 is 8.93
Starting epoch 11
time taken for epoch 11 is 9.56
Starting epoch 12
time taken for epoch 12 is 9.73
Starting epoch 13
time taken for epoch 13 is 9.46
Starting epoch 14
time taken for epoch 14 is 9.57
Starting epoch 15
train MSE after epoch 15 is 96192.92493235446
time taken for epoch 15 is 9.82
Starting epoch 16
time taken for epoch 16 is 9.87
Starting epoch 17
time taken for epoch 17 is 8.85
Starting epoch 18
time taken 

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 24289.72069592719
--------------------------------------------------
Validation:
Validation MSE 69893.86088223776


## LR = 0.02

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.02)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 6.97
Starting epoch 2
time taken for epoch 2 is 7.80
Starting epoch 3
time taken for epoch 3 is 7.15
Starting epoch 4
time taken for epoch 4 is 9.77
Starting epoch 5
train MSE after epoch 5 is 279861.3557956153
time taken for epoch 5 is 9.39
Starting epoch 6
time taken for epoch 6 is 8.43
Starting epoch 7
time taken for epoch 7 is 9.66
Starting epoch 8
time taken for epoch 8 is 9.66
Starting epoch 9
time taken for epoch 9 is 9.60
Starting epoch 10
train MSE after epoch 10 is 180470.4783257699
time taken for epoch 10 is 10.65
Starting epoch 11
time taken for epoch 11 is 9.84
Starting epoch 12
time taken for epoch 12 is 9.37
Starting epoch 13
time taken for epoch 13 is 9.71
Starting epoch 14
time taken for epoch 14 is 9.86
Starting epoch 15
train MSE after epoch 15 is 142125.53459391053
time taken for epoch 15 is 9.72
Starting epoch 16
time taken for epoch 16 is 9.32
Starting epoch 17
time taken for epoch 17 is 9.41
Starting epoch 18
time taken 

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 51018.58743673057
--------------------------------------------------
Validation:
Validation MSE 99137.40790643258


##Learning Rate Schedulers

##Constant LR

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 12.81
Starting epoch 2
time taken for epoch 2 is 12.80
Starting epoch 3
time taken for epoch 3 is 9.59
Starting epoch 4
time taken for epoch 4 is 9.12
Starting epoch 5
train MSE after epoch 5 is 241132.9766242507
time taken for epoch 5 is 9.19
Starting epoch 6
time taken for epoch 6 is 8.07
Starting epoch 7
time taken for epoch 7 is 9.15
Starting epoch 8
time taken for epoch 8 is 9.20
Starting epoch 9
time taken for epoch 9 is 8.07
Starting epoch 10
train MSE after epoch 10 is 171624.97038605914
time taken for epoch 10 is 9.12
Starting epoch 11
time taken for epoch 11 is 9.43
Starting epoch 12
time taken for epoch 12 is 9.41
Starting epoch 13
time taken for epoch 13 is 9.22
Starting epoch 14
time taken for epoch 14 is 9.03
Starting epoch 15
train MSE after epoch 15 is 136841.90374155858
time taken for epoch 15 is 8.70
Starting epoch 16
time taken for epoch 16 is 9.10
Starting epoch 17
time taken for epoch 17 is 8.64
Starting epoch 18
time take

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 24381.705026025742
--------------------------------------------------
Validation:
Validation MSE 54271.27726536941


##Step LR

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 10.82
Starting epoch 2
time taken for epoch 2 is 12.03
Starting epoch 3
time taken for epoch 3 is 8.61
Starting epoch 4
time taken for epoch 4 is 9.98
Starting epoch 5
train MSE after epoch 5 is 135756.845505824
time taken for epoch 5 is 8.41
Starting epoch 6
time taken for epoch 6 is 9.44
Starting epoch 7
time taken for epoch 7 is 9.77
Starting epoch 8
time taken for epoch 8 is 9.62
Starting epoch 9
time taken for epoch 9 is 9.82
Starting epoch 10
train MSE after epoch 10 is 93703.80708934931
time taken for epoch 10 is 9.49
Starting epoch 11
time taken for epoch 11 is 8.49
Starting epoch 12
time taken for epoch 12 is 9.28
Starting epoch 13
time taken for epoch 13 is 9.96
Starting epoch 14
time taken for epoch 14 is 8.43
Starting epoch 15
train MSE after epoch 15 is 77949.92909222575
time taken for epoch 15 is 9.69
Starting epoch 16
time taken for epoch 16 is 9.45
Starting epoch 17
time taken for epoch 17 is 8.69
Starting epoch 18
time taken f

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 46093.72855681143
--------------------------------------------------
Validation:
Validation MSE 88565.2647303534


##MultiStep LR

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,9,15], gamma=0.1)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 13.66
Starting epoch 2
time taken for epoch 2 is 11.59
Starting epoch 3
time taken for epoch 3 is 9.88
Starting epoch 4
time taken for epoch 4 is 7.62
Starting epoch 5
train MSE after epoch 5 is 166703.83402953285
time taken for epoch 5 is 9.44
Starting epoch 6
time taken for epoch 6 is 8.16
Starting epoch 7
time taken for epoch 7 is 7.83
Starting epoch 8
time taken for epoch 8 is 8.40
Starting epoch 9
time taken for epoch 9 is 7.43
Starting epoch 10
train MSE after epoch 10 is 122840.78512307408
time taken for epoch 10 is 8.80
Starting epoch 11
time taken for epoch 11 is 8.15
Starting epoch 12
time taken for epoch 12 is 7.80
Starting epoch 13
time taken for epoch 13 is 8.55
Starting epoch 14
time taken for epoch 14 is 7.28
Starting epoch 15
train MSE after epoch 15 is 106369.55284651411
time taken for epoch 15 is 8.42
Starting epoch 16
time taken for epoch 16 is 7.52
Starting epoch 17
time taken for epoch 17 is 8.55
Starting epoch 18
time tak

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 72732.95918717608
--------------------------------------------------
Validation:
Validation MSE 103371.4247014309


##Exponential LR

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 7.69
Starting epoch 2
time taken for epoch 2 is 9.43
Starting epoch 3
time taken for epoch 3 is 9.59
Starting epoch 4
time taken for epoch 4 is 8.67
Starting epoch 5
train MSE after epoch 5 is 185980.78480656745
time taken for epoch 5 is 9.43
Starting epoch 6
time taken for epoch 6 is 9.43
Starting epoch 7
time taken for epoch 7 is 8.56
Starting epoch 8
time taken for epoch 8 is 9.41
Starting epoch 9
time taken for epoch 9 is 9.26
Starting epoch 10
train MSE after epoch 10 is 131504.2790074839
time taken for epoch 10 is 8.63
Starting epoch 11
time taken for epoch 11 is 9.32
Starting epoch 12
time taken for epoch 12 is 9.09
Starting epoch 13
time taken for epoch 13 is 10.27
Starting epoch 14
time taken for epoch 14 is 9.46
Starting epoch 15
train MSE after epoch 15 is 102812.3531245507
time taken for epoch 15 is 8.35
Starting epoch 16
time taken for epoch 16 is 9.34
Starting epoch 17
time taken for epoch 17 is 9.51
Starting epoch 18
time taken 

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 29471.77895711571
--------------------------------------------------
Validation:
Validation MSE 55866.94666517259


## Will be using both, constant LR and Exponential LR going forward

## Tuning Activation function

## Linear activation

In [ ]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, 786, bias=True),
        # nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(786, 512, bias=True),
        # nn.ReLU(),
        nn.Linear(512, 384, bias=True),
        # nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(384, 256, bias=True),
        # nn.ReLU(),
        nn.Linear(256, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
# scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 13.08
Starting epoch 2
time taken for epoch 2 is 12.52
Starting epoch 3
time taken for epoch 3 is 12.13
Starting epoch 4
time taken for epoch 4 is 7.95
Starting epoch 5
train MSE after epoch 5 is 334478.6421390702
time taken for epoch 5 is 7.88
Starting epoch 6
time taken for epoch 6 is 8.52
Starting epoch 7
time taken for epoch 7 is 7.48
Starting epoch 8
time taken for epoch 8 is 8.59
Starting epoch 9
time taken for epoch 9 is 7.54
Starting epoch 10
train MSE after epoch 10 is 224872.55404383468
time taken for epoch 10 is 8.49
Starting epoch 11
time taken for epoch 11 is 8.52
Starting epoch 12
time taken for epoch 12 is 7.50
Starting epoch 13
time taken for epoch 13 is 8.41
Starting epoch 14
time taken for epoch 14 is 7.38
Starting epoch 15
train MSE after epoch 15 is 181753.45210046013
time taken for epoch 15 is 9.79
Starting epoch 16
time taken for epoch 16 is 7.46
Starting epoch 17
time taken for epoch 17 is 8.48
Starting epoch 18
time tak

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 89642.38121686914
--------------------------------------------------
Validation:
Validation MSE 84298.48815632299


## RELU

In [ ]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, 786, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(786, 512, bias=True),
        nn.ReLU(),
        nn.Linear(512, 384, bias=True),
        nn.ReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(384, 256, bias=True),
        nn.ReLU(),
        nn.Linear(256, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
# scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 14.47
Starting epoch 2
time taken for epoch 2 is 11.74
Starting epoch 3
time taken for epoch 3 is 12.15
Starting epoch 4
time taken for epoch 4 is 17.60
Starting epoch 5
train MSE after epoch 5 is 167395.20336910346
time taken for epoch 5 is 13.39
Starting epoch 6
time taken for epoch 6 is 15.95
Starting epoch 7
time taken for epoch 7 is 16.75
Starting epoch 8
time taken for epoch 8 is 17.01
Starting epoch 9
time taken for epoch 9 is 15.01
Starting epoch 10
train MSE after epoch 10 is 122161.97134817169
time taken for epoch 10 is 14.54
Starting epoch 11
time taken for epoch 11 is 15.39
Starting epoch 12
time taken for epoch 12 is 14.94
Starting epoch 13
time taken for epoch 13 is 13.98
Starting epoch 14
time taken for epoch 14 is 13.69
Starting epoch 15
train MSE after epoch 15 is 98518.00080788086
time taken for epoch 15 is 13.83
Starting epoch 16
time taken for epoch 16 is 9.88
Starting epoch 17
time taken for epoch 17 is 8.72
Starting epoch

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 30233.621827098952
--------------------------------------------------
Validation:
Validation MSE 57676.51164967386


## Leaky Relu

In [ ]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, 786, bias=True),
        nn.LeakyReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(786, 512, bias=True),
        nn.LeakyReLU(),
        nn.Linear(512, 384, bias=True),
        nn.LeakyReLU(),
        # nn.Dropout(p=0.33),
        nn.Linear(384, 256, bias=True),
        nn.LeakyReLU(),
        nn.Linear(256, output_layer),
    )

  def forward(self,x):
    return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
# scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 8.82
Starting epoch 2
time taken for epoch 2 is 7.73
Starting epoch 3
time taken for epoch 3 is 8.79
Starting epoch 4
time taken for epoch 4 is 8.62
Starting epoch 5
train MSE after epoch 5 is 166473.43368354926
time taken for epoch 5 is 7.94
Starting epoch 6
time taken for epoch 6 is 12.21
Starting epoch 7
time taken for epoch 7 is 13.17
Starting epoch 8
time taken for epoch 8 is 13.12
Starting epoch 9
time taken for epoch 9 is 12.88
Starting epoch 10
train MSE after epoch 10 is 119369.33508327896
time taken for epoch 10 is 9.16
Starting epoch 11
time taken for epoch 11 is 8.57
Starting epoch 12
time taken for epoch 12 is 8.70
Starting epoch 13
time taken for epoch 13 is 7.70
Starting epoch 14
time taken for epoch 14 is 8.68
Starting epoch 15
train MSE after epoch 15 is 95610.57675859302
time taken for epoch 15 is 7.68
Starting epoch 16
time taken for epoch 16 is 9.83
Starting epoch 17
time taken for epoch 17 is 9.44
Starting epoch 18
time ta

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 27998.07028576843
--------------------------------------------------
Validation:
Validation MSE 60170.18266456874


## TanH


In [ ]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    # self.layers = nn.Sequential(
    #     nn.Flatten(),
    #     nn.Linear(input_layer, 786, bias=True),
    #     torch.tanh(),
    #     # nn.Dropout(p=0.33),
    #     nn.Linear(786, 512, bias=True),
    #     torch.tanh(),
    #     nn.Linear(512, 384, bias=True),
    #     torch.tanh(),
    #     # nn.Dropout(p=0.33),
    #     nn.Linear(384, 256, bias=True),
    #     torch.tanh(),
    #     nn.Linear(256, output_layer),
    # )

    self.fc1 = nn.Linear(input_layer, 786, bias=True)
    self.fc2 = nn.Linear(786, 512, bias=True)
    self.fc3 = nn.Linear(512, 384, bias=True)
    self.fc4 = nn.Linear(384, 256, bias=True)
    self.fc5 = nn.Linear(256, output_layer)


  def forward(self,x):
    x = torch.tanh(self.fc1(x))
    x = torch.tanh(self.fc2(x))
    x = torch.tanh(self.fc3(x))
    x = torch.tanh(self.fc4(x))
    x = self.fc5(x)

    return x

    # x= torch.tanh(x)
    # return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
# scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 12.29
Starting epoch 2
time taken for epoch 2 is 13.12
Starting epoch 3
time taken for epoch 3 is 14.16
Starting epoch 4
time taken for epoch 4 is 17.62
Starting epoch 5
train MSE after epoch 5 is 5924446.890717637
time taken for epoch 5 is 13.08
Starting epoch 6
time taken for epoch 6 is 13.00
Starting epoch 7
time taken for epoch 7 is 12.94
Starting epoch 8
time taken for epoch 8 is 11.97
Starting epoch 9
time taken for epoch 9 is 8.46
Starting epoch 10
train MSE after epoch 10 is 5800242.967583059
time taken for epoch 10 is 8.02
Starting epoch 11
time taken for epoch 11 is 10.00
Starting epoch 12
time taken for epoch 12 is 8.71
Starting epoch 13
time taken for epoch 13 is 8.26
Starting epoch 14
time taken for epoch 14 is 8.82
Starting epoch 15
train MSE after epoch 15 is 5710191.760450141
time taken for epoch 15 is 7.66
Starting epoch 16
time taken for epoch 16 is 9.99
Starting epoch 17
time taken for epoch 17 is 7.95
Starting epoch 18
time

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 5423226.631122703
--------------------------------------------------
Validation:
Validation MSE 5253062.517927454


## Softplus


In [ ]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    # self.layers = nn.Sequential(
    #     nn.Flatten(),
    #     nn.Linear(input_layer, 786, bias=True),
    #     torch.tanh(),
    #     # nn.Dropout(p=0.33),
    #     nn.Linear(786, 512, bias=True),
    #     torch.tanh(),
    #     nn.Linear(512, 384, bias=True),
    #     torch.tanh(),
    #     # nn.Dropout(p=0.33),
    #     nn.Linear(384, 256, bias=True),
    #     torch.tanh(),
    #     nn.Linear(256, output_layer),
    # )

    self.fc1 = nn.Linear(input_layer, 786, bias=True)
    self.fc2 = nn.Linear(786, 512, bias=True)
    self.fc3 = nn.Linear(512, 384, bias=True)
    self.fc4 = nn.Linear(384, 256, bias=True)
    self.fc5 = nn.Linear(256, output_layer)


  def forward(self,x):
    x = F.softplus(self.fc1(x))
    x = F.softplus(self.fc2(x))
    x = F.softplus(self.fc3(x))
    x = F.softplus(self.fc4(x))
    x = self.fc5(x)

    return x

    # x= torch.tanh(x)
    # return self.layers(x)

In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
# scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

In [ ]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 15.29
Starting epoch 2
time taken for epoch 2 is 15.05
Starting epoch 3
time taken for epoch 3 is 16.40
Starting epoch 4
time taken for epoch 4 is 12.87
Starting epoch 5
train MSE after epoch 5 is 238036.6348932362
time taken for epoch 5 is 12.83
Starting epoch 6
time taken for epoch 6 is 12.69
Starting epoch 7
time taken for epoch 7 is 12.98
Starting epoch 8
time taken for epoch 8 is 14.02
Starting epoch 9
time taken for epoch 9 is 13.16
Starting epoch 10
train MSE after epoch 10 is 167742.53330995003
time taken for epoch 10 is 13.21
Starting epoch 11
time taken for epoch 11 is 13.23
Starting epoch 12
time taken for epoch 12 is 13.24
Starting epoch 13
time taken for epoch 13 is 13.29
Starting epoch 14
time taken for epoch 14 is 13.34
Starting epoch 15
train MSE after epoch 15 is 134887.6894597569
time taken for epoch 15 is 13.34
Starting epoch 16
time taken for epoch 16 is 13.36
Starting epoch 17
time taken for epoch 17 is 13.34
Starting epoc

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 63493.21379632639
--------------------------------------------------
Validation:
Validation MSE 83224.62644703334


## Regularization

## L1


In [17]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    # self.layers = nn.Sequential(
    #     nn.Flatten(),
    #     nn.Linear(input_layer, 786, bias=True),
    #     torch.tanh(),
    #     # nn.Dropout(p=0.33),
    #     nn.Linear(786, 512, bias=True),
    #     torch.tanh(),
    #     nn.Linear(512, 384, bias=True),
    #     torch.tanh(),
    #     # nn.Dropout(p=0.33),
    #     nn.Linear(384, 256, bias=True),
    #     torch.tanh(),
    #     nn.Linear(256, output_layer),
    # )

    self.fc1 = nn.Linear(input_layer, 786, bias=True)
    self.fc2 = nn.Linear(786, 512, bias=True)
    self.fc3 = nn.Linear(512, 384, bias=True)
    self.fc4 = nn.Linear(384, 256, bias=True)
    self.fc5 = nn.Linear(256, output_layer)


  def forward(self,x):
    x = F.softplus(self.fc1(x))
    x = F.softplus(self.fc2(x))
    x = F.softplus(self.fc3(x))
    x = F.softplus(self.fc4(x))
    x = self.fc5(x)

    return x

    # x= torch.tanh(x)
    # return self.layers(x)

In [18]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
# scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

l1_lambda = 0.01

In [19]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # Compute L1 regularization loss
      l1_loss = torch.tensor(0.)
      for param in mlp.parameters():
          l1_loss += torch.norm(param, p=1)

      # Add L1 regularization term to the loss
      loss += l1_lambda * l1_loss

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1


NameError: ignored

In [20]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

NameError: ignored

## L2


In [22]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    # self.layers = nn.Sequential(
    #     nn.Flatten(),
    #     nn.Linear(input_layer, 786, bias=True),
    #     torch.tanh(),
    #     # nn.Dropout(p=0.33),
    #     nn.Linear(786, 512, bias=True),
    #     torch.tanh(),
    #     nn.Linear(512, 384, bias=True),
    #     torch.tanh(),
    #     # nn.Dropout(p=0.33),
    #     nn.Linear(384, 256, bias=True),
    #     torch.tanh(),
    #     nn.Linear(256, output_layer),
    # )

    self.fc1 = nn.Linear(input_layer, 786, bias=True)
    self.fc2 = nn.Linear(786, 512, bias=True)
    self.fc3 = nn.Linear(512, 384, bias=True)
    self.fc4 = nn.Linear(384, 256, bias=True)
    self.fc5 = nn.Linear(256, output_layer)


  def forward(self,x):
    x = F.softplus(self.fc1(x))
    x = F.softplus(self.fc2(x))
    x = F.softplus(self.fc3(x))
    x = F.softplus(self.fc4(x))
    x = self.fc5(x)

    return x

    # x= torch.tanh(x)
    # return self.layers(x)

In [23]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005, weight_decay=0.05)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
# scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

l1_lambda = 0.001

In [24]:
train_mse=[]
train_losses=[]


epochs=20

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # # Compute L1 regularization loss
      # l1_loss = torch.tensor(0.)
      # for param in mlp.parameters():
      #     l1_loss += torch.norm(param, p=1)

      # # Add L1 regularization term to the loss
      # loss += l1_lambda * l1_loss

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 9.65
Starting epoch 2
time taken for epoch 2 is 11.15
Starting epoch 3
time taken for epoch 3 is 12.46
Starting epoch 4
time taken for epoch 4 is 12.61
Starting epoch 5
train MSE after epoch 5 is 216133.24713383729
time taken for epoch 5 is 12.40
Starting epoch 6
time taken for epoch 6 is 12.46
Starting epoch 7
time taken for epoch 7 is 12.42
Starting epoch 8
time taken for epoch 8 is 12.20
Starting epoch 9
time taken for epoch 9 is 12.71
Starting epoch 10
train MSE after epoch 10 is 159728.51190883157
time taken for epoch 10 is 13.00
Starting epoch 11
time taken for epoch 11 is 13.03
Starting epoch 12
time taken for epoch 12 is 12.55
Starting epoch 13
time taken for epoch 13 is 12.39
Starting epoch 14
time taken for epoch 14 is 14.16
Starting epoch 15
train MSE after epoch 15 is 130004.26619610334
time taken for epoch 15 is 18.24
Starting epoch 16
time taken for epoch 16 is 19.30
Starting epoch 17
time taken for epoch 17 is 17.11
Starting epo

In [25]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 49890.407724704775
--------------------------------------------------
Validation:
Validation MSE 76392.7068461928


## Drop-out


In [ ]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_layer, 786, bias=True),
        nn.ReLU(),
        nn.Dropout(p=0.25),
        nn.Linear(786, 512, bias=True),
        nn.ReLU(),
        nn.Dropout(p=0.25),
        nn.Linear(512, 384, bias=True),
        nn.ReLU(),
        nn.Dropout(p=0.25),
        nn.Linear(384, 256, bias=True),
        nn.ReLU(),
        nn.Dropout(p=0.25),
        nn.Linear(256, output_layer),
    )

  #   self.fc1 = nn.Linear(input_layer, 786, bias=True)
  #   self.dropout = nn.Dropout(0.3)
  #   self.fc2 = nn.Linear(786, 512, bias=True)
  #   self.fc3 = nn.Linear(512, 384, bias=True)
  #   self.fc4 = nn.Linear(384, 256, bias=True)
  #   self.fc5 = nn.Linear(256, output_layer)


  # def forward(self,x):
  #   x = F.softplus(self.fc1(x))
  #   # x = self.dropout(x)
  #   x = F.softplus(self.fc2(x))
  #   # x = self.dropout(x)
  #   x = F.softplus(self.fc3(x))
  #   # x = self.dropout(x)
  #   x = F.softplus(self.fc4(x))
  #   # x = self.dropout(x)
  #   x = self.fc5(x)

  #   return x

    # x= torch.tanh(x)

  def forward(self, x):
    return self.layers(x)


In [ ]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

l1_lambda = 0.001

In [ ]:
train_mse=[]
train_losses=[]


epochs=100

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # # Compute L1 regularization loss
      # l1_loss = torch.tensor(0.)
      # for param in mlp.parameters():
      #     l1_loss += torch.norm(param, p=1)

      # # Add L1 regularization term to the loss
      # loss += l1_lambda * l1_loss

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 12.69
Starting epoch 2
time taken for epoch 2 is 11.90
Starting epoch 3
time taken for epoch 3 is 10.19
Starting epoch 4
time taken for epoch 4 is 11.77
Starting epoch 5
train MSE after epoch 5 is 285529.5373291202
time taken for epoch 5 is 10.82
Starting epoch 6
time taken for epoch 6 is 7.94
Starting epoch 7
time taken for epoch 7 is 7.05
Starting epoch 8
time taken for epoch 8 is 7.97
Starting epoch 9
time taken for epoch 9 is 6.91
Starting epoch 10
train MSE after epoch 10 is 232906.80386804283
time taken for epoch 10 is 8.12
Starting epoch 11
time taken for epoch 11 is 7.11
Starting epoch 12
time taken for epoch 12 is 8.17
Starting epoch 13
time taken for epoch 13 is 7.67
Starting epoch 14
time taken for epoch 14 is 7.55
Starting epoch 15
train MSE after epoch 15 is 204391.6218024524
time taken for epoch 15 is 8.17
Starting epoch 16
time taken for epoch 16 is 7.18
Starting epoch 17
time taken for epoch 17 is 8.59
Starting epoch 18
time ta

In [ ]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

NameError: ignored

## No Drop-out


In [20]:
input_layer = 1387
# hidden_layer=256
output_layer=1

class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    # self.layers = nn.Sequential(
    #     nn.Flatten(),
    #     nn.Linear(input_layer, 786, bias=True),
    #     torch.tanh(),
    #     # nn.Dropout(p=0.33),
    #     nn.Linear(786, 512, bias=True),
    #     torch.tanh(),
    #     nn.Linear(512, 384, bias=True),
    #     torch.tanh(),
    #     # nn.Dropout(p=0.33),
    #     nn.Linear(384, 256, bias=True),
    #     torch.tanh(),
    #     nn.Linear(256, output_layer),
    # )

    self.fc1 = nn.Linear(input_layer, 786, bias=True)
    self.dropout = nn.Dropout(0.3)
    self.fc2 = nn.Linear(786, 512, bias=True)
    self.fc3 = nn.Linear(512, 384, bias=True)
    self.fc4 = nn.Linear(384, 256, bias=True)
    self.fc5 = nn.Linear(256, output_layer)


  def forward(self,x):
    x = F.softplus(self.fc1(x))
    # x = self.dropout(x)
    x = F.softplus(self.fc2(x))
    # x = self.dropout(x)
    x = F.softplus(self.fc3(x))
    # x = self.dropout(x)
    x = F.softplus(self.fc4(x))
    # x = self.dropout(x)
    x = self.fc5(x)

    return x

    # x= torch.tanh(x)
    # return self.layers(x)

In [21]:
mlp=MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
criterion = nn.MSELoss()

l1_lambda = 0.001

In [ ]:
train_mse=[]
train_losses=[]


epochs=80

total_start_time=time.time()
# Run the training loop
for epoch in range(epochs): # Testing the epochs between 50 and 100

    # Start timing the training process
    start_time = time.time()

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data

    for i, data in enumerate(train_loader, 0):

      # Get inputs
      inputs, targets = data

      # Zero the gradients
      optimizer.zero_grad()

      # Perform forward pass
      outputs = mlp(inputs)
      # outputs.to(device)

      # Compute loss
      loss = criterion(outputs.flatten(), targets)

      # # Compute L1 regularization loss
      # l1_loss = torch.tensor(0.)
      # for param in mlp.parameters():
      #     l1_loss += torch.norm(param, p=1)

      # # Add L1 regularization term to the loss
      # loss += l1_lambda * l1_loss

      # Perform backward pass
      loss.backward()

      # Perform optimization
      optimizer.step()

      # Print statistics
      current_loss += loss.item()
      if i % 500 == 499:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

    train_losses.append(loss.item())

    scheduler.step()

    for input_batch, target_batch in train_loader:
      predict_batch = mlp(input_batch)
      mse = mean_squared_error(predict_batch.flatten().tolist(), target_batch.flatten().tolist())
      train_mse.append(mse)

    if (epoch+1)%5==0:
        print(f'train MSE after epoch {epoch+1} is {np.mean(train_mse)}')

    end_time=time.time()

    print(f'time taken for epoch {epoch+1} is {end_time-start_time:.2f}')

# Process is complete.
total_end_time=time.time()
print(f'Training process has finished. Time taken is {total_end_time-total_start_time}')

Starting epoch 1
time taken for epoch 1 is 13.15
Starting epoch 2
time taken for epoch 2 is 12.49
Starting epoch 3
time taken for epoch 3 is 9.95
Starting epoch 4
time taken for epoch 4 is 11.03
Starting epoch 5
train MSE after epoch 5 is 346859.86231641803
time taken for epoch 5 is 12.00
Starting epoch 6
time taken for epoch 6 is 12.51
Starting epoch 7
time taken for epoch 7 is 20.87
Starting epoch 8
time taken for epoch 8 is 18.73
Starting epoch 9
time taken for epoch 9 is 15.71
Starting epoch 10
train MSE after epoch 10 is 249567.7824194771
time taken for epoch 10 is 12.50
Starting epoch 11
time taken for epoch 11 is 12.68
Starting epoch 12
time taken for epoch 12 is 13.57
Starting epoch 13
time taken for epoch 13 is 13.60
Starting epoch 14
time taken for epoch 14 is 12.45
Starting epoch 15
train MSE after epoch 15 is 192286.24783450973
time taken for epoch 15 is 9.67
Starting epoch 16
time taken for epoch 16 is 9.19
Starting epoch 17
time taken for epoch 17 is 9.51
Starting epoch 1

In [24]:
# sets the model in evaluation mode
mlp.eval()
train_mse=[]

# Train dataset
for input_batch, target_batch in train_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  train_mse.append(mse)


print('Training:')
print(f'Train MSE {np.mean(train_mse)}')

print('--------------------------------------------------')

valid_mse=[]
# Vaidation dataset
for input_batch, target_batch in val_loader:
  predict_batch = mlp(input_batch)
  mse=mean_squared_error(predict_batch.flatten().tolist(), target_batch.tolist())
  valid_mse.append(mse)

print('Validation:')
print(f'Validation MSE {np.mean(valid_mse)}')

Training:
Train MSE 11273.353606946099
--------------------------------------------------
Validation:
Validation MSE 53967.017957855394


## Creating a custom class for test data

In [25]:
class CustomDataset_test(Dataset):
    def __init__(self, filename):
        """
        This is extraction step of the data processing
        """
        file_out = pd.read_csv(filename)
        x = file_out.iloc[:,:].values
        sc = StandardScaler()
        x =  sc.fit_transform(x)

        """
        This is the transformation step of data
        """
        self.X = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx]




In [26]:
test_set = CustomDataset_test('test.csv')

<ipython-input-25-9a11ae79d439>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y, dtype=torch.float32)


In [27]:
# Loading the dataset using data loader and making mini-batches
test_loader = DataLoader(
    dataset=test_set,
    batch_size=64,
    shuffle=False
)

In [34]:
for input_batch in test_loader:
  predict_batch_test = mlp(input_batch)

In [35]:
predict_batch = torch.clamp(predict_batch, min=0)

In [ ]:
predict_batch

In [39]:
predict_batch_list = predict_batch.flatten().tolist()

In [44]:
len(test_set)

7825

In [62]:
test_list=[]
for x in range(len(test_set)):
  test_list.append(test_set[x][0].tolist())

In [65]:
test_list[1:5]

[-0.8635117411613464,
 1.7192877531051636,
 -1.0914406776428223,
 0.1690276712179184]

In [66]:
final_df = pd.DataFrame(zip(test_list, predict_batch_list), columns=['Id','y'])

In [67]:
final_df.to_csv('final_df.csv',sep=',', index=False)

